In [13]:
import pickle
import nltk
import os
from pyvi import ViTokenizer
from nltk.tag import pos_tag 
import string
from tqdm import tqdm
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, GRU, Dense, Input, Add, Concatenate, Reshape, Lambda, BatchNormalization, Dropout, Embedding, LSTM
from keras.optimizers import SGD, RMSprop
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from underthesea import ner
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from time import time
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Bidirectional
from keras.callbacks import TensorBoard
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [198]:
#Do e k kiếm được data trên mạng nên em tự tạo data dùng ner() của thư viện underthesea
file_list = []
for (dirpath, dirname, filename) in os.walk('Train_Full/'):
    for f in filename:
        file_list.append(dirpath+'/'+f)

In [201]:
import re
def clean_document(doc):
    clean_text = re.sub(r'([^\s\w]|_)+\n', '', doc)
    return clean_text.strip()

In [202]:
sequences = []
for f in file_list[:3000]: 
    f1 = open(f, encoding='utf-16')
    doc = f1.read()
    doc = doc.split('. ')
    for i in doc:
        i = clean_document(i)
        sequences.append(i)
sequences[:5]

['Apollo tổ chức cuộc thi tìm hiểu giáo dục Singapore (NLĐ) - Tổ chức giáo dục và đào tạo Apollo Việt Nam vừa thông báo tổ chức cuộc thi “Tìm hiểu giáo dục Singapore” dành cho HS-SV từ 15 đến 23 tuổi, hiện đang sống tại Hà Nội và TPHCM',
 'Để tìm hiểu nội dung cuộc thi, xin đến Trung tâm Giáo dục Apollo (số 26 Phùng Khắc Khoan, quận 1 - TPHCM hoặc số 67 Lê Văn Hưu - TP Hà Nội) hoặc truy cập vào website: www.apolloedutrain.com và đăng ký tham dự',
 'Giải thưởng gồm 4 giải nhất là 4 chuyến du lịch Singapore, 4 giải nhì là học bổng tiếng Anh giao tiếp, 4 giải ba là học bổng luyện ngữ âm tiếng Anh',
 'Hạn chót đăng ký tham dự 18-11-2005',
 'TPHCM đào tạo đại học cho người sau cai nghiện Đại học Mở-Bán công đã khai giảng lớp đại học từ xa khoa Xã hội và Nhân văn đầu tiên cho học viên, người sau cai nghiện và một số cán bộ, công nhân viên các trung tâm giáo dục dạy nghề của thành phốLớp học, được đào tạo theo chế độ tín chỉ, gồm 169 sinh viên, trong đó có 47 học viên sau cai nghiện']

In [205]:
def preprocess(sent):
    sent = ner(sent)
    return sent

In [206]:
preprocess('apollo tổ chức cuộc thi tìm hiểu giáo dục singapore nlđ  tổ chức giáo dục và đào tạo apollo việt nam vừa thông báo tổ chức cuộc thi tìm hiểu giáo dục singapore dành cho hssv từ 15 đến 23 tuổi hiện đang sống tại hà nội và tphcm')

[('apollo', 'N', 'B-NP', 'O'),
 ('tổ chức', 'V', 'B-VP', 'O'),
 ('cuộc', 'N', 'B-NP', 'O'),
 ('thi', 'V', 'B-VP', 'O'),
 ('tìm hiểu', 'V', 'B-VP', 'O'),
 ('giáo dục', 'N', 'B-NP', 'O'),
 ('singapore', 'V', 'B-VP', 'O'),
 ('nlđ', 'N', 'B-NP', 'O'),
 ('tổ chức', 'V', 'B-VP', 'O'),
 ('giáo dục', 'V', 'B-VP', 'O'),
 ('và', 'C', 'O', 'O'),
 ('đào tạo', 'V', 'B-VP', 'O'),
 ('apollo', 'N', 'B-NP', 'O'),
 ('việt', 'N', 'B-NP', 'O'),
 ('nam', 'N', 'B-NP', 'O'),
 ('vừa', 'R', 'O', 'O'),
 ('thông báo', 'V', 'B-VP', 'O'),
 ('tổ chức', 'V', 'B-VP', 'O'),
 ('cuộc', 'N', 'B-NP', 'O'),
 ('thi', 'V', 'B-VP', 'O'),
 ('tìm hiểu', 'V', 'B-VP', 'O'),
 ('giáo dục', 'N', 'B-NP', 'O'),
 ('singapore', 'N', 'B-NP', 'O'),
 ('dành', 'V', 'B-VP', 'O'),
 ('cho', 'E', 'B-PP', 'O'),
 ('hssv', 'N', 'B-NP', 'O'),
 ('từ', 'E', 'B-PP', 'O'),
 ('15', 'M', 'B-NP', 'O'),
 ('đến', 'E', 'B-PP', 'O'),
 ('23', 'M', 'B-NP', 'O'),
 ('tuổi', 'N', 'B-NP', 'O'),
 ('hiện', 'N', 'B-NP', 'O'),
 ('đang', 'R', 'O', 'O'),
 ('sống', 'V', '

In [207]:
f = open('train.txt','w',encoding='utf-8')
for i in sequences:
    for a,b,c,d in ner(i):
        f.write(a.replace(' ','_')+' '+b+' '+c+' '+d+'\n')

In [3]:
import import_ipynb
from helper import load_glove_dict, get_glove_vector, accuracy, entity_count, one_hot_encode, one_hot_decode
import nltk
from sklearn.metrics import precision_recall_fscore_support

importing Jupyter notebook from helper.ipynb


In [4]:
#Do trong hàm ner() hiển thị sent, pos, chunk và entity nên mục tiêu của hàm này cho out về 4 cái
# Hàm này còn có tác dụng gộp I-ORG và B-ORG thành ORG, tương tự cho LOC và PER
def conll_sentences(conll_file):
    temp_sent = []
    temp_pos = []
    temp_chunk = []
    temp_entity = []
    with open(conll_file,'r',encoding='utf-8') as f:
        conll_raw_data = f.readlines()
    for line in conll_raw_data:
        if line != '':
            split_line = line.split()
            if len(split_line) == 4:
                temp_sent.append(split_line[0])
                temp_pos.append(split_line[1])
                temp_chunk.append(split_line[2])
                old_ent = split_line[3]
                if old_ent in ('I-ORG', 'B-ORG'):
                    new_ent = 'ORG'
                elif old_ent in ('I-LOC', 'B-LOC'):
                    new_ent = 'LOC'
                elif old_ent in ('I-PER', 'B-PER'):
                    new_ent = 'PER'
                else:
                    new_ent = 'O'
                temp_entity.append(new_ent)
    
    return temp_sent, temp_pos, temp_chunk, temp_entity


In [5]:
# Từ tập data đầu vào, lấy word và entity
train_words, _, _, train_entities = conll_sentences('train.txt')
testa_words, _, _, testa_entities = conll_sentences('testa.txt')
#testb_words, _, _, testb_entities = conll_sentences('testb.txt')

In [6]:
print(train_words[:10])
print(train_entities[:10])

['Apollo', 'tổ_chức', 'cuộc', 'thi', 'tìm_hiểu', 'giáo_dục', 'Singapore', '(', 'NLĐ', ')']
['ORG', 'O', 'O', 'O', 'O', 'O', 'LOC', 'O', 'O', 'O']


In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_words)

In [8]:
tokenizer.fit_on_texts(train_entities)

In [9]:
tokenizer.word_index

{'o': 1,
 'loc': 2,
 'per': 3,
 'và': 4,
 'có': 5,
 'các': 6,
 'của': 7,
 'là': 8,
 'được': 9,
 'cho': 10,
 'công': 11,
 'trong': 12,
 'không': 13,
 'đã': 14,
 'học': 15,
 'một': 16,
 'người': 17,
 'với': 18,
 'những': 19,
 'đến': 20,
 'ở': 21,
 'để': 22,
 'tại': 23,
 'nước': 24,
 'năm': 25,
 'về': 26,
 'bộ': 27,
 'này': 28,
 'trường': 29,
 'từ': 30,
 'phải': 31,
 'sẽ': 32,
 'đồng': 33,
 'nhà': 34,
 'sinh': 35,
 'thành': 36,
 'ngày': 37,
 'số': 38,
 'ra': 39,
 'trên': 40,
 'việc': 41,
 'theo': 42,
 'cũng': 43,
 'làm': 44,
 'dân': 45,
 'hiện': 46,
 'vào': 47,
 'nhiều': 48,
 'chỉ': 49,
 'ông': 50,
 'định': 51,
 'như': 52,
 'khi': 53,
 'chính': 54,
 'đầu': 55,
 'động': 56,
 'đường': 57,
 'hội': 58,
 'xe': 59,
 'đó': 60,
 'bị': 61,
 'nhân': 62,
 'nam': 63,
 'trình': 64,
 'thông': 65,
 'thể': 66,
 'viên': 67,
 'lại': 68,
 'còn': 69,
 'gia': 70,
 '2': 71,
 'quốc': 72,
 'lý': 73,
 'trung': 74,
 'thì': 75,
 '1': 76,
 'quan': 77,
 'văn': 78,
 'nhưng': 79,
 'cơ': 80,
 'dự': 81,
 'đi': 82,
 'thi'

In [10]:
sequences_digit = tokenizer.texts_to_sequences(train_words)
print(sequences_digit[:100])
sequences_entities = tokenizer.texts_to_sequences(train_entities)
print(sequences_entities[:100])

[[2552], [148, 107], [312], [83], [394, 680], [155, 275], [822], [], [758], [], [], [148, 107], [155, 275], [4], [242, 173], [2552], [84, 63], [288], [65, 143], [148, 107], [312], [83], [112], [394, 680], [155, 275], [822], [118], [621], [10], [882, 1044], [30], [400], [20], [831], [424], [], [46], [134], [333], [23], [128, 120], [4], [598], [22], [394, 680], [120, 660], [312], [83], [], [502], [20], [74, 162], [155, 275, 2552], [], [38], [1020], [1831, 759, 1308], [], [212], [76], [], [598], [293], [38], [2390], [448, 78, 1662], [], [86], [128, 120], [], [293], [1374, 973], [47], [1684], [], [1191, 7697, 1459], [4], [359, 257], [222, 81], [154, 834], [576], [164], [154], [121], [8], [164], [706], [129, 190], [822], [], [164], [154], [2092], [8], [15, 652], [304], [124], [114, 159], [], [164]]
[[183], [1], [1], [1], [1], [1], [2], [1], [1], [1], [1], [1], [1], [1], [1], [3], [3], [1], [1], [1], [1], [1], [1], [1], [2], [2], [1], [1], [1], [3], [1], [1], [1], [1], [1], [1], [1], [1], [1

In [11]:
import numpy as np
import tensorflow as tf

# Dùng RaggedTensor vì khi preprocessing em đã bỏ các kí tự đặc biệt, nên khi đọc văn bản, các kí tự đặc biệt không có trong word_index thì sẽ có mảng rỗng
# Nên khi convert to tensor sẽ bị lỗi non-rectangular Python sequence to Tensor
# Tác dụng của Ragged sẽ biến những non-rectangular thành [0]

vocab_size = len(tokenizer.word_index) + 1
#X = tf.data.Dataset.from_generator(lambda: iter(zip(sequences_digit, sequences_entities)),output_types=(tf.int64,tf.int64))
#y = tf.data.Dataset.from_generator(sequences_entities,output_types=tf.int64)
X = tf.ragged.constant(sequences_digit)
X = X.to_tensor()
y = tf.ragged.constant(sequences_entities)
y = y.to_tensor()
#y = np.asarray(sequences_entities).astype('object') 
#keras.utils.to_categorical(sequences_entities, num_classes=vocab_size)


In [14]:
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=5))
model.add(BatchNormalization())
model.add(LSTM(512, return_sequences=True))
model.add(LSTM(512))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(1, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 5)              82755     
_________________________________________________________________
batch_normalization (BatchNo (None, 5, 5)              20        
_________________________________________________________________
lstm (LSTM)                  (None, 5, 512)            1060864   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense (Dense)                (None, 100)               51300     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 100)               4

In [15]:
# model em đã fit 100 lần và đã lưu accuracy khoảng 0.88
model.fit(X, y, batch_size=512, epochs=2)

Epoch 1/2
  11/2241 [..............................] - ETA: 54:51 - loss: 0.0000e+00 - accuracy: 0.8837

KeyboardInterrupt: 

In [16]:
# Xử lý tương tự cho data_set và accuracy tầm khoảng 0.879
sequences_digit_test = tokenizer.texts_to_sequences(testa_words)
sequences_entities_test = tokenizer.texts_to_sequences(testa_entities)
X_test = tf.ragged.constant(sequences_digit_test)
X_test = X_test.to_tensor()
y_test = tf.ragged.constant(sequences_entities_test)
y_test = y_test.to_tensor()
# X_test = pad_sequences(sequences_digit,5)
# y_test = sequences_entities #keras.utils.to_categorical(sequences_entities, num_classes=vocab_size)
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy ",accuracy)

Testing Accuracy  0.8798899054527283


In [17]:
import re
from pyvi import ViTokenizer, ViPosTagger
def clean_document(doc):
    clean_text = ViTokenizer.tokenize(doc)
    clean_text = re.sub(r'([^\s\w]|_)+\n', '', clean_text)
    return clean_text.split()

In [18]:
# VỚi đầu vào là text, xử lý preprocessing, sau đó dựa vào model để đưa ra kết quả, với từng result trong yy ta dựa vào word_index trả về enity
def excute(text):
    text = clean_document(text)
    print(text)
    sequences_digit_of_text = tokenizer.texts_to_sequences(text)
    X_of_text = tf.ragged.constant(sequences_digit_of_text)
    X_of_text = X_of_text.to_tensor()
    yy = model.predict_classes(X_of_text)
    out = []
    for token in yy:
        for word, index in tokenizer.word_index.items():
            if index == token:
                out.append(word)
                break
    return out

In [21]:
text = 'Ca covid ở TPHCM ngày càng tăng cáo buộc nhà nước phải có những quyết định đúng đắn'
print(excute(text))
# Do data đầu vào của em dựa vào thư viện có sẵn, nên độ chính xác dựa trên data đầu vào, và entity khớp vẫn đúng, nhưng đôi khi sẽ bị sai

['Ca', 'covid', 'ở', 'TPHCM', 'ngày_càng', 'tăng', 'cáo_buộc', 'nhà_nước', 'phải', 'có', 'những', 'quyết_định', 'đúng_đắn']
['O', 'O', 'O', 'LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
